In [1]:
%run helper_functions.ipynb

In [2]:
import pandas as pd

In [3]:
%%time

transactions = pd.read_parquet('data/transactions_train.parquet')
customers = pd.read_parquet('data/customers.parquet')
articles = pd.read_parquet('data/articles.parquet')

# sample = 0.05
# transactions = pd.read_parquet(f'data/transactions_train_sample_{sample}.parquet')
# customers = pd.read_parquet(f'data/customers_sample_{sample}.parquet')
# articles = pd.read_parquet(f'data/articles_train_sample_{sample}.parquet')

CPU times: user 1.71 s, sys: 558 ms, total: 2.27 s
Wall time: 748 ms


In [4]:
test_week = transactions.week.max() + 1
transactions = transactions[transactions.week > transactions.week.max() - 10]

# Generating candidates

### Last purchase candidates

In [5]:
%%time

c2weeks = transactions.groupby('customer_id')['week'].unique()

CPU times: user 12.3 s, sys: 692 ms, total: 13 s
Wall time: 12.2 s


In [6]:
%%time

c2weeks2shifted_weeks = {}

for c_id, weeks in c2weeks.items():
    c2weeks2shifted_weeks[c_id] = {}
    for i in range(weeks.shape[0]-1):
        c2weeks2shifted_weeks[c_id][weeks[i]] = weeks[i+1]
    c2weeks2shifted_weeks[c_id][weeks[-1]] = test_week

CPU times: user 407 ms, sys: 44.3 ms, total: 451 ms
Wall time: 451 ms


In [7]:
candidates_last_purchase = transactions.copy()

In [8]:
%%time

weeks = []
for i, (c_id, week) in enumerate(zip(transactions['customer_id'], transactions['week'])):
    weeks.append(c2weeks2shifted_weeks[c_id][week])
    
candidates_last_purchase.week=weeks

CPU times: user 5.82 s, sys: 24.2 ms, total: 5.85 s
Wall time: 5.85 s


### Bestsellers candidates

In [9]:
mean_price = transactions \
    .groupby(['week', 'article_id'])['price'].mean()

In [10]:
sales = transactions \
    .groupby('week')['article_id'].value_counts() \
    .groupby('week').rank(method='dense', ascending=False) \
    .groupby('week').head(12).rename('bestseller_rank').astype('int8')

In [11]:
bestsellers_previous_week = pd.merge(sales, mean_price, on=['week', 'article_id']).reset_index()
bestsellers_previous_week.week += 1

In [12]:
unique_transactions = transactions \
    .groupby(['week', 'customer_id']) \
    .head(1) \
    .drop(columns=['article_id', 'price']) \
    .copy()

In [13]:
candidates_bestsellers = pd.merge(
    unique_transactions,
    bestsellers_previous_week,
    on='week',
)

In [14]:
test_set_transactions = unique_transactions.drop_duplicates('customer_id').reset_index(drop=True)
test_set_transactions.week = test_week

In [15]:
candidates_bestsellers_test_week = pd.merge(
    test_set_transactions,
    bestsellers_previous_week,
    on='week'
)

In [16]:
candidates_bestsellers = pd.concat([candidates_bestsellers, candidates_bestsellers_test_week])
candidates_bestsellers.drop(columns='bestseller_rank', inplace=True)

# Combining transactions and candidates / negative examples

In [17]:
transactions['purchased'] = 1

In [18]:
data = pd.concat([transactions, candidates_last_purchase, candidates_bestsellers])
data.purchased.fillna(0, inplace=True)

In [19]:
data.drop_duplicates(['customer_id', 'article_id', 'week'], inplace=True)

In [20]:
data.purchased.mean()

0.13607582749165664

### Add bestseller information

In [21]:
data = pd.merge(
    data,
    bestsellers_previous_week[['week', 'article_id', 'bestseller_rank']],
    on=['week', 'article_id'],
    how='left'
)

In [22]:
data = data[data.week != data.week.min()]
data.bestseller_rank.fillna(999, inplace=True)

In [23]:
data = pd.merge(data, articles, on='article_id', how='left')
data = pd.merge(data, customers, on='customer_id', how='left')

In [24]:
data.sort_values(['week', 'customer_id'], inplace=True)
data.reset_index(drop=True, inplace=True)

In [25]:
train = data[data.week != test_week]
test = data[data.week==test_week].drop_duplicates(['customer_id', 'article_id', 'sales_channel_id']).copy()

In [26]:
train_baskets = train.groupby(['week', 'customer_id'])['article_id'].count().values

In [27]:
columns_to_use = ['article_id', 'product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id',
'perceived_colour_master_id', 'department_no', 'index_code',
'index_group_no', 'section_no', 'garment_group_no', 'FN', 'Active',
'club_member_status', 'fashion_news_frequency', 'age', 'postal_code', 'bestseller_rank']

In [28]:
%%time

train_X = train[columns_to_use]
train_y = train['purchased']

test_X = test[columns_to_use]

CPU times: user 169 ms, sys: 67.8 ms, total: 237 ms
Wall time: 236 ms


# Model training

In [29]:
from lightgbm.sklearn import LGBMRanker

In [30]:
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    n_estimators=1,
    importance_type='gain',
    verbose=10
)

In [31]:
%%time

ranker = ranker.fit(
    train_X,
    train_y,
    group=train_baskets,
)

[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.849662
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.153183
[LightGBM] [Debug] init for col-wise cost 0.111599 seconds, init for row-wise cost 0.272583 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.172005 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 1083
[LightGBM] [Info] Number of data points in the train set: 11381612, number of used features: 18
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
CPU times: user 39.9 s, sys: 871 ms, total: 40.7 s
Wall time: 2.28 s


In [32]:
for i in ranker.feature_importances_.argsort()[::-1]:
    print(columns_to_use[i], ranker.feature_importances_[i]/ranker.feature_importances_.sum())

bestseller_rank 0.9989735967378934
age 0.00021946832092321984
article_id 0.00018145568808180745
garment_group_no 0.00015314425097510742
product_type_no 0.00013122713021882564
section_no 7.067155512680504e-05
postal_code 6.750257936664963e-05
department_no 6.690380100526352e-05
club_member_status 6.454133710213458e-05
colour_group_code 5.372958935864729e-05
perceived_colour_value_id 1.775900994807456e-05
fashion_news_frequency 0.0
Active 0.0
FN 0.0
index_code 0.0
perceived_colour_master_id 0.0
graphical_appearance_no 0.0
index_group_no 0.0


# Calculate predictions

In [33]:
%time

test['preds'] = ranker.predict(test_X)

c_id2predicted_article_ids = test \
    .sort_values(['customer_id', 'preds'], ascending=False) \
    .groupby('customer_id')['article_id'].apply(list).to_dict()

bestsellers_last_week = \
    bestsellers_previous_week[bestsellers_previous_week.week == bestsellers_previous_week.week.max()]['article_id'].tolist()

CPU times: user 73 µs, sys: 5 µs, total: 78 µs
Wall time: 7.63 µs


# Create submission

In [34]:
sub = pd.read_csv('data/sample_submission.csv')

In [35]:
%%time
preds = []
for c_id in customer_hex_id_to_int(sub.customer_id):
    pred = c_id2predicted_article_ids.get(c_id, [])
    pred = pred + bestsellers_last_week
    preds.append(pred[:12])

CPU times: user 3.29 s, sys: 75.8 ms, total: 3.37 s
Wall time: 3.37 s


In [36]:
preds = [' '.join(['0' + str(p) for p in ps]) for ps in preds]
sub.prediction = preds

In [37]:
sub_name = 'basic_model_submission'
sub.to_csv(f'data/subs/{sub_name}.csv.gz', index=False)

In [38]:
!kaggle competitions submit -c h-and-m-personalized-fashion-recommendations -f 'data/subs/{sub_name}.csv.gz' -m {sub_name}

100%|██████████████████████████████████████| 57.9M/57.9M [00:30<00:00, 2.02MB/s]
Successfully submitted to H&M Personalized Fashion Recommendations